In [1]:
import nltk
import string
from nltk.wsd import lesk
from nltk.corpus import stopwords, wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from rake_nltk import Rake, Metric

In [2]:
from datasets import load_dataset
squad = load_dataset("squad")
text = squad["train"]["context"][0]

d:\learnx.ai\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [155]:
rake_nltk = Rake(max_length=3, 
                include_repeated_phrases = False)

rake_nltk.extract_keywords_from_text(text)
keywords = rake_nltk.get_ranked_phrases()

filtered_keywords = set()
#punctuation removal from the text followed
for i in keywords:
    i = i.translate(str.maketrans("","",string.punctuation)).strip()
    filtered_keywords.add(i)    


In [164]:
import random

sample = random.sample(filtered_keywords, 5)
masked_sentence = []
cased_keywords = ([(i.upper(), i.lower(), i.capitalize(), i.title()) for i in sample])
temp = text

for i,j,k,l in cased_keywords:
  temp = temp.replace(i,"[MASK]").replace(j,"[MASK]").replace(k,"[MASK]").replace(l,"[MASK]")

for i in nltk.sent_tokenize(temp):
  if "[MASK]" in i:
    masked_sentence.append(i)

In [195]:
for i,j in enumerate(masked_sentence):
    print("{:}{:10}{:}".format(i," ",j))

0          [MASK] the [MASK]'s gold dome is a golden statue of the Virgin Mary.
1          Immediately in front of the [MASK] and facing it, is a copper statue of Christ with [MASK] with the legend "Venite Ad Me Omnes".
2          Next to the [MASK] is the Basilica of the [MASK].
3          At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, [MASK] of Mary.


In [196]:
#Keyword extraction using rake-nltk
rake_nltk = Rake(max_length = 1, 
                include_repeated_phrases = False,
                punctuations = string.punctuation)

rake_nltk.extract_keywords_from_text(text)
keywords = rake_nltk.get_ranked_phrases()[:10]

#punctuation and stopwords removal from the text followed by lemmatization
stop_words = set(stopwords.words("english"))
text_no_punc = text.translate(str.maketrans("","",string.punctuation))
word_tokens = word_tokenize(text_no_punc.lower())
#filtered_sentence = [w for w in word_tokens if not w in stop_words]
lemmatizer = WordNetLemmatizer()
lemmatized_sentence = [lemmatizer.lemmatize(w) for w in word_tokens]

In [197]:
wn_keywords = []
wn_definitions = []
try:
    for i in keywords:
        wn_keywords.append(i)
        wn_definitions.append(lesk(lemmatized_sentence, i).definition())
except AttributeError:
    pass

In [198]:
import random

sample = random.sample(wn_keywords, len(wn_keywords)-1)
try:
    for i in range(len(sample)):
        print("{:100}{:10}".format(sample[i], wn_definitions[i]))
except AttributeError:
    pass

simple                                                                                              a person lacking intelligence or common sense
school                                                                                              the period of instruction in a school; the time period when school is in session
prayer                                                                                              copy that is not the original; something that has been copied
mary                                                                                                (mathematics) a transformation in which the direction of one axis is reversed
next                                                                                                the act of communicating with a deity (especially as a petition or in adoration or contrition or thanksgiving)
replica                                                                                             at the time or occasio

In [19]:
from transformers import AutoTokenizer, BertForPreTraining

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = BertForPreTraining.from_pretrained(checkpoint)

In [20]:
import torch
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)
x = outputs.prediction_logits[0]
y = outputs.seq_relationship_logits
ids = torch.argmax(x, dim=1)
tokenizer.decode(ids)

'.., my dog is..'